In [12]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
import math
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks, peak_prominences, resample_poly
from sklearn import svm
from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score
from multiprocessing import Pool
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm
from scipy import signal


Read in data

In [13]:
data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')

# data = data[['ax','ay', 'az', 'mx', 'my', 'mz', 'gx', 'gy', 'gz','altitude']]
data = data[['ax','ay', 'az', 'mx', 'my', 'mz', 'gx', 'gy', 'gz', 'phone_ax', 'phone_ay', 'phone_az','phone_gx', 'phone_gy', 'phone_gz', 'phone_mx', 'phone_my', 'phone_mz','altitude']]
#magneto sampling rate is ~12.5 10 sample averaging
#accel gyro sampling rate is 200 100 sample averaging
#10 sample 

In [ ]:
d = Recording("data/train/train_trace_000.pkl")
for i in range(1, len(d.data['gx'].timestamps)-1):
    print(f" acc {1/(d.data['phone_ax'].timestamps[i]-d.data['phone_ax'].timestamps[i-1])}")
    print(f" gyro {1/(d.data['phone_gx'].timestamps[i]-d.data['phone_gx'].timestamps[i-1])}")
    print(f" mag {1/(d.data['phone_mx'].timestamps[i]-d.data['phone_mx'].timestamps[i-1])}")


Define filtering and vector projection functions

In [14]:
def window_average(input, winlen: int):
#downsample/filter with an average, if the input is 40 elements long and winlen is 10 output will have 4 elements

    numwins = int(len(input)/winlen)
    remainder = len(input)%winlen
    output=[]

    for i in range(numwins):
        output.append(np.mean(input[winlen*i:winlen*(i+1)]))
    
    return output

#average accelerometer and magnetometer data based on the trace number
def get_filtered_data(numtrace = 100, downsample = False, filter=False, source="watch"):
    if source == "watch":
        axl = "ax"; ayl = "ay"; azl = "az"
        gxl = "gx"; gyl = "gy"; gzl = "gz"
        mxl = "mx"; myl = "my"; mzl = "mz"
    elif source == "phone":
        axl = "phone_ax"; ayl = "phone_ay"; azl = "phone_az"
        gxl = "phone_gx"; gyl = "phone_gy"; gzl = "phone_gz"
        mxl = "phone_mx"; myl = "phone_my"; mzl = "phone_mz"


    ax=[];ay=[];ax=[];gx=[];gy=[];gz=[];mx=[];my=[];mz=[]
    if downsample:
        down, up = (len(data[axl].loc[numtrace].values)/len(data[mxl].loc[numtrace].values)).as_integer_ratio()
        
        ax = resample_poly(data[axl].loc[numtrace].values, up, down)
        ay = resample_poly(data[ayl].loc[numtrace].values, up, down)
        az = resample_poly(data[azl].loc[numtrace].values, up, down)
        
        gx = resample_poly(data[gxl].loc[numtrace].values, up, down)
        gy = resample_poly(data[gyl].loc[numtrace].values, up, down)
        gz = resample_poly(data[gzl].loc[numtrace].values, up, down)

        if filter:
            sos = signal.cheby2(2, 20, [0.5, 3.125], 'bandpass', fs=12.5, output='sos')

            ax = signal.sosfiltfilt(sos,ax)
            ay = signal.sosfiltfilt(sos,ay)
            az = signal.sosfiltfilt(sos,az)

            gx = signal.sosfiltfilt(sos,gx)
            gy = signal.sosfiltfilt(sos,gy)
            gz = signal.sosfiltfilt(sos,gz)

            mx = signal.sosfiltfilt(sos,data[mxl].loc[numtrace].values)
            my = signal.sosfiltfilt(sos,data[myl].loc[numtrace].values)
            mz = signal.sosfiltfilt(sos,data[mzl].loc[numtrace].values)
        else:
            mx = data[mxl].loc[numtrace].values
            my = data[myl].loc[numtrace].values
            mz = data[mzl].loc[numtrace].values

    else:
        #some arrays don't have the same lengths, equalize to the shortest
        
        ax = data[axl].loc[numtrace].values
        ay = data[ayl].loc[numtrace].values
        az = data[azl].loc[numtrace].values

        gx = data[gxl].loc[numtrace].values
        gy = data[gyl].loc[numtrace].values
        gz = data[gzl].loc[numtrace].values

        mx = data[mxl].loc[numtrace].values
        my = data[myl].loc[numtrace].values
        mz = data[mzl].loc[numtrace].values

        mini = min(len(ax), len(ay))
        mini = min(mini, len(az))
        mini = min(mini, len(gx))
        mini = min(mini, len(gy))
        mini = min(mini, len(gz))
        mini = min(mini, len(mx))
        mini = min(mini, len(my))
        mini = min(mini, len(mz))

        if len(ax) > mini:
            ax = ax[:-(len(ax)-mini)]
        if len(ay) > mini:
            ay = ay[:-(len(ay)-mini)]
        if len(az) > mini:
            az = az[:-(len(az)-mini)]


        if len(gx) > mini:
            gx = gx[:-(len(gx)-mini)]
        if len(gy) > mini:
            gy = gy[:-(len(gy)-mini)]
        if len(gz) > mini:
            gz = gz[:-(len(gz)-mini)]


        if len(mx) > mini:
            mx = mx[:-(len(mx)-mini)]
        if len(my) > mini:
            my = my[:-(len(my)-mini)]
        if len(mz) > mini:
            mz = mz[:-(len(mz)-mini)]

    # print(f"{len(ax)} {len(ay)} {len(az)} {len(gx)} {len(gy)} {len(gz)} {len(mx)} {len(my)} {len(mz)}")

    return (ax, ay, az, gx, gy, gz, mx, my, mz)

def getproj(a,b):
    return a - ((np.dot(a, b) / np.dot(b, b)) * b)

def getbearing(accelvec, magvec):
    proj = getproj(magvec, accelvec)
    return (math.atan2(proj[1], proj[0]) * 180.0 / math.pi ) + 180.0

def get_trace_bearings(numtrace):
    bearings = []
    ax, ay, az, gx, gy, gz, mx, my, mz = get_filtered_data(numtrace)
    limit = min(len(ax), len(mx))
    for i in range(limit):
        accelvec = np.array([az[i], ay[i], ax[i]])
        magvec   = np.array([mz[i], my[i], mx[i]])
        bearings.append(getbearing(accelvec, magvec))
    return bearings

#test the bearing function
ax, ay, az, gx, gy, gz, mx, my, mz = get_filtered_data(100, True, True)

accelvec = np.array([az[0], ay[0], ax[0]])
magvec   = np.array([mz[0], my[0], mx[0]])

print(getbearing(accelvec, magvec))

265.7231512699832


In [15]:
class Madgwick:
    def __init__(self, gyr: np.ndarray = None, acc: np.ndarray = None, mag: np.ndarray = None, **kwargs):
        self.gyr: np.ndarray = gyr
        self.acc: np.ndarray = acc
        self.mag: np.ndarray = mag
        self.frequency: float = kwargs.get('frequency', 100.0)
        self.Dt: float = kwargs.get('Dt', (1.0/self.frequency) if self.frequency else 0.01)
        self.q0: np.ndarray = kwargs.get('q0')
        self._set_gain(**kwargs)
        self._assert_validity_of_inputs()
        if self.acc is not None and self.gyr is not None:
            self.Q: np.ndarray = self._compute_all()

    def _set_gain(self, **kwargs) -> None:
        """Set the gain parameter."""
        self.gain_imu: float = kwargs.get('gain_imu', 0.033)
        self.gain_marg: float = kwargs.get('gain_marg', 0.041)
        self.gain: float = kwargs.get('beta')  # Setting gain with `beta` will be removed in the future.
        if self.gain is None:
            self.gain: float = kwargs.get('gain', self.gain_imu if self.mag is None else self.gain_marg)

    def _assert_validity_of_inputs(self):
        """Asserts the validity of the inputs."""
        for item in ["frequency", "Dt", "gain", "gain_imu", "gain_marg"]:
            if isinstance(self.__getattribute__(item), bool):
                raise TypeError(f"Parameter '{item}' must be numeric.")
            if not isinstance(self.__getattribute__(item), (int, float)):
                raise TypeError(f"Parameter '{item}' is not a non-zero number.")
            if self.__getattribute__(item) <= 0.0:
                raise ValueError(f"Parameter '{item}' must be a non-zero number.")
        if self.q0 is not None:
            if not isinstance(self.q0, (list, tuple, np.ndarray)):
                raise TypeError(f"Parameter 'q0' must be an array. Got {type(self.q0)}.")
            self.q0 = np.copy(self.q0)
            if self.q0.shape != (4,):
                raise ValueError(f"Parameter 'q0' must be an array of shape (4,). It is {self.q0.shape}.")
            if not np.allclose(np.linalg.norm(self.q0), 1.0):
                raise ValueError(f"Parameter 'q0' must be a versor (norm equal to 1.0). Its norm is equal to {np.linalg.norm(self.q0)}.")


    def _compute_all(self) -> np.ndarray:
        """
        Estimate the quaternions given all data.

        Attributes ``gyr`` and ``acc`` must contain data. If ``mag`` contains
        data, the updateMARG() method is used.

        Returns
        -------
        Q : numpy.ndarray
            M-by-4 Array with all estimated quaternions, where M is the number
            of samples.

        """
        self.gyr = np.copy(self.gyr)
        self.acc = np.copy(self.acc)
        if self.acc.shape != self.gyr.shape:
            raise ValueError("acc and gyr are not the same size")
        num_samples = len(self.acc)
        Q = np.zeros((num_samples, 4))
        # Compute with IMU architecture
        if self.mag is None:
            Q[0] = acc2q(self.acc[0]) if self.q0 is None else self.q0/np.linalg.norm(self.q0)
            for t in range(1, num_samples):
                Q[t] = self.updateIMU(Q[t-1], self.gyr[t], self.acc[t])
            return Q
        # Compute with MARG architecture
        self.mag = np.copy(self.mag)
        if self.mag.shape != self.gyr.shape:
            raise ValueError("mag and gyr are not the same size")
        Q[0] = ecompass(self.acc[0], self.mag[0], frame='NED', representation='quaternion')
        for t in range(1, num_samples):
            Q[t] = self.updateMARG(Q[t-1], self.gyr[t], self.acc[t], self.mag[t])
        return Q
        
    def updateMARG(self, q: np.ndarray, gyr: np.ndarray, acc: np.ndarray, mag: np.ndarray, dt: float = None) -> np.ndarray:

        # q : numpy.ndarray
        #     A-priori quaternion.
        # gyr : numpy.ndarray
        #     Sample of tri-axial Gyroscope in rad/s
        # acc : numpy.ndarray
        #     Sample of tri-axial Accelerometer in m/s^2
        # mag : numpy.ndarray
        #     Sample of tri-axial Magnetometer in nT
        # dt : float, default: None
        #     Time step, in seconds, between consecutive Quaternions.

        dt = self.Dt if dt is None else dt
        if gyr is None or not np.linalg.norm(gyr) > 0:
            return q
        if mag is None or not np.linalg.norm(mag) > 0:
            return self.updateIMU(q, gyr, acc)
        qDot = 0.5 * q_prod(q, [0, *gyr])                           # (eq. 12)
        a_norm = np.linalg.norm(acc)
        if a_norm > 0:
            a = acc/a_norm
            m = mag/np.linalg.norm(mag)
            # Rotate normalized magnetometer measurements
            h = q_prod(q, q_prod([0, *m], q_conj(q)))               # (eq. 45)
            bx = np.linalg.norm([h[1], h[2]])                       # (eq. 46)
            bz = h[3]
            qw, qx, qy, qz = q/np.linalg.norm(q)
            # Objective function (eq. 31)
            f = np.array([2.0*(qx*qz - qw*qy)   - a[0],
                            2.0*(qw*qx + qy*qz)   - a[1],
                            2.0*(0.5-qx**2-qy**2) - a[2],
                            2.0*bx*(0.5 - qy**2 - qz**2) + 2.0*bz*(qx*qz - qw*qy)       - m[0],
                            2.0*bx*(qx*qy - qw*qz)       + 2.0*bz*(qw*qx + qy*qz)       - m[1],
                            2.0*bx*(qw*qy + qx*qz)       + 2.0*bz*(0.5 - qx**2 - qy**2) - m[2]])
            # Jacobian (eq. 32)
            J = np.array([[-2.0*qy,               2.0*qz,              -2.0*qw,               2.0*qx             ],
                            [ 2.0*qx,               2.0*qw,               2.0*qz,               2.0*qy             ],
                            [ 0.0,                 -4.0*qx,              -4.0*qy,               0.0                ],
                            [-2.0*bz*qy,            2.0*bz*qz,           -4.0*bx*qy-2.0*bz*qw, -4.0*bx*qz+2.0*bz*qx],
                            [-2.0*bx*qz+2.0*bz*qx,  2.0*bx*qy+2.0*bz*qw,  2.0*bx*qx+2.0*bz*qz, -2.0*bx*qw+2.0*bz*qy],
                            [ 2.0*bx*qy,            2.0*bx*qz-4.0*bz*qx,  2.0*bx*qw-4.0*bz*qy,  2.0*bx*qx          ]])
            gradient = J.T@f                                        # (eq. 34)
            gradient /= np.linalg.norm(gradient)
            qDot -= self.gain*gradient                              # (eq. 33)
        q_new = q + qDot*dt                                         # (eq. 13)
        q_new /= np.linalg.norm(q_new)
        return q_new
    
    def updateIMU(self, q: np.ndarray, gyr: np.ndarray, acc: np.ndarray, dt: float = None) -> np.ndarray:
        """
        Quaternion Estimation with IMU architecture.

        Parameters
        ----------
        q : numpy.ndarray
            A-priori quaternion.
        gyr : numpy.ndarray
            Sample of tri-axial Gyroscope in rad/s
        acc : numpy.ndarray
            Sample of tri-axial Accelerometer in m/s^2
        dt : float, default: None
            Time step, in seconds, between consecutive Quaternions.

        Returns
        -------
        q : numpy.ndarray
            Estimated quaternion.
        """
        dt = self.Dt if dt is None else dt
        if gyr is None or not np.linalg.norm(gyr) > 0:
            return q
        qDot = 0.5 * q_prod(q, [0, *gyr])                           # (eq. 12)
        a_norm = np.linalg.norm(acc)
        if a_norm > 0:
            a = acc/a_norm
            qw, qx, qy, qz = q/np.linalg.norm(q)
            # Objective function (eq. 25)
            f = np.array([2.0*(qx*qz - qw*qy)   - a[0],
                          2.0*(qw*qx + qy*qz)   - a[1],
                          2.0*(0.5-qx**2-qy**2) - a[2]])
            if np.linalg.norm(f) > 0:
                # Jacobian (eq. 26)
                J = np.array([[-2.0*qy,  2.0*qz, -2.0*qw, 2.0*qx],
                              [ 2.0*qx,  2.0*qw,  2.0*qz, 2.0*qy],
                              [ 0.0,    -4.0*qx, -4.0*qy, 0.0   ]])
                # Objective Function Gradient
                gradient = J.T@f                                    # (eq. 34)
                gradient /= np.linalg.norm(gradient)
                qDot -= self.gain*gradient                          # (eq. 33)
        q_new = q + qDot*dt                                         # (eq. 13)
        q_new /= np.linalg.norm(q_new)
        return q_new

def ecompass(a: np.ndarray, m: np.ndarray, frame: str = 'ENU', representation: str = 'rotmat') -> np.ndarray:
    """
    Orientation from accelerometer and magnetometer readings

    Parameters
    ----------
    a : numpy.ndarray
        Sample of tri-axial accelerometer, in m/s^2.
    m : numpy.ndarray
        Sample of tri-axial magnetometer, in uT.
    frame : str, default: ``'ENU'``
        Local tangent plane coordinate frame.
    representation : str, default: ``'rotmat'``
        Orientation representation. Options are: ``'rotmat'``, ``'quaternion'``,
        ``'rpy'``, ``'axisangle'``.

    Returns
    -------
    np.ndarray
        Estimated orientation.

    Raises
    ------
    ValueError
        When wrong local tangent plane coordinates, or invalid representation,
        is given.
    """
    if frame.upper() not in ['ENU', 'NED']:
        raise ValueError("Wrong local tangent plane coordinate frame. Try 'ENU' or 'NED'")
    if representation.lower() not in ['rotmat', 'quaternion', 'rpy', 'axisangle']:
        raise ValueError("Wrong representation type. Try 'rotmat', 'quaternion', 'rpy', or 'axisangle'")
    a = np.copy(a)
    m = np.copy(m)
    if a.shape[-1] != 3 or m.shape[-1] != 3:
        raise ValueError("Input vectors must have exactly 3 elements.")
    m /= np.linalg.norm(m)
    Rz = a/np.linalg.norm(a)
    if frame.upper() == 'NED':
        Ry = np.cross(Rz, m)
        Rx = np.cross(Ry, Rz)
    else:
        Rx = np.cross(m, Rz)
        Ry = np.cross(Rz, Rx)
    Rx /= np.linalg.norm(Rx)
    Ry /= np.linalg.norm(Ry)
    R = np.c_[Rx, Ry, Rz].T
    if representation.lower() == 'quaternion':
        return chiaverini(R)
    if representation.lower() == 'rpy':
        phi = np.arctan2(R[1, 2], R[2, 2])    # Roll Angle
        theta = -np.arcsin(R[0, 2])           # Pitch Angle
        psi = np.arctan2(R[0, 1], R[0, 0])    # Yaw Angle
        return np.array([phi, theta, psi])
    if representation.lower() == 'axisangle':
        angle = np.arccos((R.trace()-1)/2)
        axis = np.zeros(3)
        if angle != 0:
            S = np.array([R[2, 1]-R[1, 2], R[0, 2]-R[2, 0], R[1, 0]-R[0, 1]])
            axis = S/(2*np.sin(angle))
        return (axis, angle)
    return R

def chiaverini(dcm: np.ndarray) -> np.ndarray:
    dcm = np.copy(dcm)
    if dcm.ndim not in [2, 3]:
        raise ValueError('dcm must be a 2- or 3-dimensional array.')
    if dcm.shape[-2:] != (3, 3):
        raise ValueError(f"dcm must be an array of shape 3-by-3 or N-by-3-by-3. Got {dcm.shape}")
    if dcm.ndim < 3:
        q = np.zeros(4)
        q[0] = 0.5*np.sqrt(np.clip(dcm.trace(), -1.0, 3.0) + 1.0)
        q[1] = 0.5*np.sign(dcm[2, 1]-dcm[1, 2])*np.sqrt(np.clip(dcm[0, 0]-dcm[1, 1]-dcm[2, 2], -1.0, 1.0)+1.0)
        q[2] = 0.5*np.sign(dcm[0, 2]-dcm[2, 0])*np.sqrt(np.clip(dcm[1, 1]-dcm[2, 2]-dcm[0, 0], -1.0, 1.0)+1.0)
        q[3] = 0.5*np.sign(dcm[1, 0]-dcm[0, 1])*np.sqrt(np.clip(dcm[2, 2]-dcm[0, 0]-dcm[1, 1], -1.0, 1.0)+1.0)
        if not any(q):
            q[0] = 1.0
        q /= np.linalg.norm(q)
        return q
    Q = np.zeros((dcm.shape[0], 4))
    Q[:, 0] = 0.5*np.sqrt(np.clip(dcm.trace(axis1=1, axis2=2), -1.0, 3.0) + 1.0)
    Q[:, 1] = 0.5*np.sign(dcm[:, 2, 1] - dcm[:, 1, 2])*np.sqrt(np.clip(dcm[:, 0, 0]-dcm[:, 1, 1]-dcm[:, 2, 2], -1.0, 1.0) + 1.0)
    Q[:, 2] = 0.5*np.sign(dcm[:, 0, 2] - dcm[:, 2, 0])*np.sqrt(np.clip(dcm[:, 1, 1]-dcm[:, 2, 2]-dcm[:, 0, 0], -1.0, 1.0) + 1.0)
    Q[:, 3] = 0.5*np.sign(dcm[:, 1, 0] - dcm[:, 0, 1])*np.sqrt(np.clip(dcm[:, 2, 2]-dcm[:, 0, 0]-dcm[:, 1, 1], -1.0, 1.0) + 1.0)
    Q /= np.linalg.norm(Q, axis=1)[:, None]
    return Q

def q_prod(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    pq = np.zeros(4)
    pq[0] = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    pq[1] = p[0]*q[1] + p[1]*q[0] + p[2]*q[3] - p[3]*q[2]
    pq[2] = p[0]*q[2] - p[1]*q[3] + p[2]*q[0] + p[3]*q[1]
    pq[3] = p[0]*q[3] + p[1]*q[2] - p[2]*q[1] + p[3]*q[0]
    return pq

def q_conj(q: np.ndarray) -> np.ndarray:
    q = np.copy(q)
    if q.ndim > 2 or q.shape[-1] != 4:
        raise ValueError(f"Quaternion must be of shape (4,) or (N, 4), but has shape {q.shape}")
    return np.array([1., -1., -1., -1.])*np.array(q)

def q2euler(q: np.ndarray) -> np.ndarray:
    """
    Euler Angles from unit Quaternion.

    Parameters
    ----------
    q : numpy.ndarray
        Quaternion

    Returns
    -------
    angles : numpy.ndarray
        Euler Angles around X-, Y- and Z-axis.

    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles#Quaternion_to_Euler_Angles_Conversion

    """
    if sum(np.array([1., 0., 0., 0.])-q) == 0.0:
        return np.zeros(3)
    if len(q) != 4:
        return None
    R_00 = 2.0*q[0]**2 - 1.0 + 2.0*q[1]**2
    R_10 = 2.0*(q[1]*q[2] - q[0]*q[3])
    R_20 = 2.0*(q[1]*q[3] + q[0]*q[2])
    R_21 = 2.0*(q[2]*q[3] - q[0]*q[1])
    R_22 = 2.0*q[0]**2 - 1.0 + 2.0*q[3]**2
    #rotation around x, roll
    phi = np.arctan2( R_21, R_22)
    #rotation around y, pitch
    theta = -np.arctan( R_20/np.sqrt(1.0-R_20**2))
    #rotation around z, yaw
    psi = np.arctan2( R_10, R_00)
    return np.array([phi, theta, psi])

def acc2q(a: np.ndarray, return_euler: bool = False) -> np.ndarray:
    """
    Quaternion from given acceleration.

    Parameters
    ----------
    a : numpy.ndarray
        A sample of 3 orthogonal accelerometers.
    return_euler : bool, default: False
        Return pose as Euler angles

    Returns
    -------
    pose : numpy.ndarray
        Quaternion or Euler Angles.
    """
    q = np.array([1.0, 0.0, 0.0, 0.0])
    ex, ey, ez = 0.0, 0.0, 0.0
    if np.linalg.norm(a) > 0 and len(a) == 3:
        ax, ay, az = a
        # Normalize accelerometer measurements
        a_norm = np.linalg.norm(a)
        ax /= a_norm
        ay /= a_norm
        az /= a_norm
        # Euler Angles from Gravity vector
        ex = np.arctan2(ay, az)
        ey = np.arctan2(-ax, np.sqrt(ay**2 + az**2))
        ez = 0.0
        if return_euler:
            return np.array([ex, ey, ez])*RAD2DEG
        # Euler to Quaternion
        cx2 = np.cos(ex/2.0)
        sx2 = np.sin(ex/2.0)
        cy2 = np.cos(ey/2.0)
        sy2 = np.sin(ey/2.0)
        q = np.array([cx2*cy2, sx2*cy2, cx2*sy2, -sx2*sy2])
        q /= np.linalg.norm(q)
    return q

In [133]:
#116, path 3, walking only
# {'path_idx': 4, 'activities': [1], 'step_count': None, 'watch_loc': 0} 115
# {'path_idx': 3, 'activities': [1], 'step_count': None, 'watch_loc': 2} 116
# {'path_idx': 1, 'activities': [1], 'step_count': None, 'watch_loc': 0} 117
# {'path_idx': 2, 'activities': [1], 'step_count': None, 'watch_loc': 2} 120
# {'path_idx': 1, 'activities': [1], 'step_count': None, 'watch_loc': 2} 121
# {'path_idx': 0, 'activities': [1], 'step_count': None, 'watch_loc': 2} 122
# {'path_idx': 3, 'activities': [1], 'step_count': None, 'watch_loc': 0} 123
# {'path_idx': 0, 'activities': [1], 'step_count': None, 'watch_loc': 0} 125
for i in range(100, 130):
    d = Recording(f"data/train/train_trace_{i}.pkl")
    if(d.labels['activities']==[1]):
        print(f"{d.labels} {i}")

{'path_idx': 2, 'activities': [1], 'step_count': None, 'watch_loc': 0} 103
{'path_idx': 2, 'activities': [1], 'step_count': None, 'watch_loc': 0} 106
{'path_idx': 4, 'activities': [1], 'step_count': None, 'watch_loc': 1} 107
{'path_idx': 4, 'activities': [1], 'step_count': None, 'watch_loc': 2} 109
{'path_idx': 0, 'activities': [1], 'step_count': None, 'watch_loc': 0} 112
{'path_idx': 4, 'activities': [1], 'step_count': None, 'watch_loc': 0} 115
{'path_idx': 3, 'activities': [1], 'step_count': None, 'watch_loc': 2} 116
{'path_idx': 1, 'activities': [1], 'step_count': None, 'watch_loc': 0} 117
{'path_idx': 2, 'activities': [1], 'step_count': None, 'watch_loc': 2} 120
{'path_idx': 1, 'activities': [1], 'step_count': None, 'watch_loc': 2} 121
{'path_idx': 0, 'activities': [1], 'step_count': None, 'watch_loc': 2} 122
{'path_idx': 3, 'activities': [1], 'step_count': None, 'watch_loc': 0} 123
{'path_idx': 0, 'activities': [1], 'step_count': None, 'watch_loc': 0} 125
{'path_idx': 1, 'activiti

In [ ]:

ntrace = 116
include_mag = True

def madgwick_headings(num_trace, source="watch"):
    if(source=="watch"):
        ax, ay, az, gx, gy, gz, mx, my, mz = get_filtered_data(num_trace, downsample=True, filter=False, source=source)
        #magnetometer has lower fs and everything downsampled to that
        samplerate = 12.5
    else:
        ax, ay, az, gx, gy, gz, mx, my, mz = get_filtered_data(num_trace, downsample=False, filter=False, source=source)
        #everything has the same sample rate
        samplerate = 100
        
    # if len(ax) != len(ay) or len(ax) != len(az) or len(az) != len(ay):
    #     print(f"{num_trace} orospu cocugu array")

    # if len(gx) != len(gy) or len(gx) != len(gz) or len(gz) != len(gy):
    #     print(f"{num_trace} orospu cocugu array")
    
    # if len(mx) != len(my) or len(mx) != len(mz) or len(mz) != len(my):
    #     print(f"{num_trace} orospu cocugu array")

    acc_data  = np.concatenate([np.array(az).reshape(-1,1),np.array(ay).reshape(-1,1),np.array(ax).reshape(-1,1)], axis=1)
    gyro_data = np.concatenate([np.array(gz).reshape(-1,1),np.array(gy).reshape(-1,1),np.array(gx).reshape(-1,1)], axis=1)
    mag_data  = np.concatenate([np.array(mz).reshape(-1,1),np.array(my).reshape(-1,1),np.array(mx).reshape(-1,1)], axis=1)

    
    madgwick = Madgwick(gyr=gyro_data, acc=acc_data, mag=mag_data, frequency=samplerate, gain=0.038)

    # 

    # fig = plt.figure()
    # ax = plt.axes(projection="3d")


    current = [0, 0, 0]
    x = []
    y = [] 
    z = []

    from scipy.spatial.transform import Rotation
    limit = len(madgwick.Q)
    # limit=100
    for i in (range(limit)):
        euler = q2euler(madgwick.Q[i]) *180/math.pi
        rot = Rotation.from_euler('xyz', euler)
        #rotate unit vector to get cartesian headings, x=0
        vector = np.array(rot.as_matrix()).dot(np.array([0,0,1]))

        # plt.arrow(current[0], current[1], vector[0], vector[1])
        current += vector
        x.append(current[0])
        y.append(current[1])
        z.append(current[2])

    x = np.array(x)
    y = np.array(y)
    z = np.array(z)

    return x, y, z

def plot_trajectory(x, y, title):
    avg = 2000

    u = np.diff(window_average(x,avg))
    v = np.diff(window_average(y,avg))
    print(len(u))
    print(len(v))

    pos_x = window_average(x,avg)[:-1] + u/2
    pos_y = window_average(y,avg)[:-1] + v/2
    norm = np.sqrt(u**2+v**2) 

    fig, ax = plt.subplots()
    ax.plot(window_average(x,avg),window_average(y,avg), marker="o")
    ax.quiver(pos_x, pos_y, u/norm, v/norm, angles="xy", zorder=5, pivot="mid")
    
    plt.title(title)
    plt.show()

x, y, z = madgwick_headings(ntrace, source="phone")
plt.scatter(x,y,z)
plt.show()
# plot_trajectory(x, z, "xz")
# plot_trajectory(x, y, "xy")
# plot_trajectory(z, y, "zy")

# x, y, z = madgwick_headings(ntrace, source="watch")

# plot_trajectory(x, z, "xz")
# plot_trajectory(x, y, "xy")
# plot_trajectory(z, y, "zy")


# x = np.arange(0, len(data['altitude'].loc[ntrace].values), 1)

# m,b = np.polyfit(x, data['altitude'].loc[ntrace].values, 1)
# poly1d_fn = np.poly1d((m,b))
# plt.plot(x,data['altitude'].loc[ntrace].values, 'yo', x, poly1d_fn(x), '--k')

# plt.title(f'altitude {np.mean(data['altitude'].loc[ntrace].values[-100:]) - np.mean(data['altitude'].loc[ntrace].values[:100])} {m*1000} ')
# plt.show()

Plot bearings as a sanity check

In [ ]:
#plot bearings extracted from a plot
# z,y,x

for trace in range(5):
    ax, ay, az, gx, gy, gz, mx, my, mz = get_filtered_data(trace, downsample=True)
    current = [0,0,0]
    limit = min(len(ax), len(mx))
    # limit=20
    bearings = []
    for point in range(limit):
        accelvec = np.array([az[point], ay[point], ax[point]])
        magvec   = np.array([mz[point], my[point], mx[point]])

        proj = getproj(magvec, accelvec)
        bearings.append(getbearing(accelvec, magvec))
        # length = math.sqrt(proj[0]**2 + proj[1]**2)
        length =3
        plt.arrow(current[0], current[1], proj[0]/length, proj[1]/length)
        current += proj
    print(bearings)
    plt.title(f"trace {trace}")
    plt.show()


In [17]:
#classify the number of bearings per direction based on the trace number
def classify_bearings(source = "phone", numtrace=0):
    FEATURES = ['avgdist2d', 'avgdist3d', "dist1", "dist2", "dist3", "dist4", "dist5"]
    for idx, feature in enumerate(FEATURES):
        FEATURES[idx] = FEATURES[idx] + f'_{source}' 

    avgdist2d=0; avgdist3d=0;
    try: 
        x, y, z = madgwick_headings(numtrace, source=source)
        # if source == "phone":
        #     x = window_average(x,50)
        #     y = window_average(y,50)
        #     z = window_average(z,50)

        # else:
        #     x = window_average(x,5)
        #     y = window_average(y,5)
        #     z = window_average(z,5) 
             
        avgdist2d=math.sqrt((x[-1]+x[0])**2 + (y[-1]+y[0])**2 )
        avgdist3d=math.sqrt((x[-1]+x[0])**2 + (y[-1]+y[0])**2 + (z[-1]+z[0])**2)

        seglen = int(len(x)/5)
        dist1 = math.sqrt((x[seglen-1]   - x[0]       )**2   +   (y[seglen-1]   - y[0]       )**2)
        dist2 = math.sqrt((x[seglen*2-1] - x[seglen]  )**2   +   (y[seglen*2-1] - y[seglen]  )**2)
        dist3 = math.sqrt((x[seglen*3-1] - x[seglen*2])**2   +   (y[seglen*3-1] - y[seglen*2])**2)
        dist4 = math.sqrt((x[seglen*4-1] - x[seglen*3])**2   +   (y[seglen*4-1] - y[seglen*3])**2)
        dist5 = math.sqrt((x[seglen*5-1] - x[seglen*4])**2   +   (y[seglen*5-1] - y[seglen*4])**2)
    except:
        print(f"except {numtrace}")
        # DistanceFeatures = ['north', 'northeast', "east", 'southeast', 'south', 'southwest', 'west', 'northwest']
        DistanceFeatures = ['north', "east", 'south',  'west']
        for segment in range(5):
            for idx, feature in enumerate(DistanceFeatures):
                FEATURES.append(DistanceFeatures[idx] + f'_seg{segment}_{source}')
        #7 common features + 8 per segment features * 5 segments = 47 zeros
        df_features = pd.DataFrame(index = [FEATURES], data = np.zeros(7 + len(DistanceFeatures) * 5))
        df_features = pd.DataFrame.transpose(df_features)
        df_features.columns = df_features.columns.map(''.join)
        return df_features 


    df_features = pd.DataFrame(index = [FEATURES], data = [avgdist2d,avgdist3d, dist1, dist2, dist3, dist4, dist5])
    df_features = pd.DataFrame.transpose(df_features)
    df_features.columns = df_features.columns.map(''.join)

    #divide data into 5 parts and classify headings separately
    # print(f"total length {len(x)}")
    north = 0; northeast  = 0; east = 0; southeast = 0; south = 0; southwest = 0; west = 0; northwest = 0;
    DistanceFeatures = ['north', "east", 'south',  'west']
    for i in range(len(x)):
        bearing = (math.atan2(x[segment*seglen + i], z[segment*seglen + i]) * 180.0 / math.pi ) + 180.0

        if bearing <= 90:
            north += 1
        elif bearing <= 180:
            east += 1
        elif bearing <= 270:
            south += 1
        else:
            west += 1

        # seg_features = pd.DataFrame(index = [DistanceFeatures], data = [north, northeast, east, southeast, south, southwest, west, northwest])
    seg_features = pd.DataFrame(index = [DistanceFeatures], data = [north, east,  south,  west])
    seg_features = pd.DataFrame.transpose(seg_features)
    seg_features.columns = seg_features.columns.map(''.join)
    df_features = pd.concat([df_features, seg_features],axis=1)

    return df_features


#wrapper functions because to make using map easier
def classify_bearings_watch(numtrace):
    return classify_bearings(source = "watch", numtrace=numtrace)


def classify_bearings_phone(numtrace):
    return classify_bearings(source = "phone", numtrace=numtrace)

def classify_all_watch_recordings():
    nums = np.arange(0,len(data['ax']), 1)
    with Pool() as p:
        directions = p.map(classify_bearings_watch, nums)
    directions = pd.concat(directions, ignore_index=True)    
    
    return directions

def classify_all_phone_recordings():
    nums = np.arange(0,len(data['phone_ax']), 1)
    with Pool() as p:
        directions = p.map(classify_bearings_phone, nums)
    directions = pd.concat(directions, ignore_index=True)    
    
    return directions

watch_bearing_directions = classify_all_watch_recordings()
watch_bearing_directions.to_pickle(path='data/watch_bearing_directions.pkl.zst', compression={'method': 'zstd'})
watch_bearing_directions

phone_bearing_directions = classify_all_phone_recordings()
phone_bearing_directions.to_pickle(path='data/phone_bearing_directions.pkl.zst', compression={'method': 'zstd'})
phone_bearing_directions


/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid value encountered in divide
  m /= np.linalg.norm(m)
/tmp/ipykernel_390906/179091411.py:205: RuntimeWarning: invalid 

except 133


,avgdist2d_phone,avgdist3d_phone,dist1_phone,dist2_phone,dist3_phone,dist4_phone,dist5_phone,north_seg0_phone,east_seg0_phone,south_seg0_phone,...,south_seg2_phone,west_seg2_phone,north_seg3_phone,east_seg3_phone,south_seg3_phone,west_seg3_phone,north_seg4_phone,east_seg4_phone,south_seg4_phone,west_seg4_phone
0,701.262034,701.914399,714.744140,198.662799,180.951272,117.861469,60.025861,2.0,343.0,10187.0,...,8521.0,3171.0,0.0,0.0,2931.0,8761.0,0.0,0.0,9545.0,2147.0
1,546.100524,639.582683,279.686353,176.365972,304.562859,351.965020,137.860149,4048.0,5848.0,20.0,...,1380.0,6238.0,0.0,0.0,0.0,10313.0,0.0,0.0,0.0,10313.0
2,273.543447,295.314831,30.013414,46.503873,129.985201,281.760226,152.275170,6937.0,1405.0,97.0,...,5867.0,0.0,1026.0,7978.0,2533.0,346.0,192.0,980.0,9782.0,929.0
3,227.103638,261.311774,131.317905,17.261247,40.955429,111.597296,87.701581,0.0,0.0,10390.0,...,10390.0,0.0,0.0,0.0,10390.0,0.0,0.0,0.0,10390.0,0.0
4,122.890822,395.853094,123.932715,172.087942,70.076674,369.222708,401.084197,51.0,623.0,6417.0,...,0.0,12385.0,183.0,0.0,0.0,12205.0,355.0,0.0,0.0,12033.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,262.714781,857.006496,100.557999,514.759911,223.413368,51.560612,35.946617,100.0,0.0,5359.0,...,6510.0,0.0,0.0,0.0,6510.0,0.0,0.0,0.0,6510.0,0.0
392,72.041355,351.148629,24.321658,118.623628,70.291726,117.637710,88.201234,2930.0,27.0,563.0,...,0.0,12355.0,50.0,0.0,0.0,12305.0,1575.0,0.0,0.0,10780.0
393,313.503377,452.199550,155.971067,192.170330,96.291299,171.615609,241.675099,2364.0,46.0,0.0,...,0.0,44.0,2215.0,12738.0,0.0,0.0,0.0,3372.0,11581.0,0.0
394,341.469218,346.729092,53.490086,65.357879,15.527700,130.874747,232.507608,6981.0,2033.0,38.0,...,0.0,7351.0,1328.0,0.0,0.0,7793.0,3936.0,0.0,0.0,5185.0


In [ ]:
for i in range(147, 200):
    d = Recording(f"data/train/train_trace_{i}.pkl")
    sos = signal.cheby2(1, 20, 0.5, 'lowpass', fs=12.5, output='sos')
    alt = signal.sosfiltfilt(sos,d.data["altitude"].values)
    plt.plot(alt)
    plt.title(f"{i} {np.var(alt)}")
    plt.show()